In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np


C:\Users\SMest\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SMest\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\SMest\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import glob
from ipfx import feature_extractor
import os
import pyabf
from IPython.display import clear_output
import pandas as pd
import pyAPisolation.abf_featureextractor as abfe

In [3]:

abf_list_dir = glob.glob("*.abf")
abf_list = [pyabf.ABF(x) for x in abf_list_dir]
abf_list_name = [str(x) for x in abf_list_dir]
pairs = [c for c in zip(abf_list_name, abf_list)]
test=feature_extractor.SpikeFeatureExtractor()
print(abf_list)

[ABFcore(abf="C:\Users\SMest\source\repos\smestern\pyABF-APisolation\pyAPisolation\bin\20o01023.abf", loadData=True), ABFcore(abf="C:\Users\SMest\source\repos\smestern\pyABF-APisolation\pyAPisolation\bin\20o01033.abf", loadData=True)]


In [5]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

    
textbox_style = {'width':'50%', 'description_width': 'initial' }   
    
class abfanalysis(widgets.Box):
     
    def __init__(self):
        super().__init__()
        output = widgets.Output()
        
        plt.close('all') 
        with output:
            self.fig, self.abf_plot = plt.subplots(constrained_layout=True, figsize=(10, 7.5))
        self.abf_list = abf_list
        self.pairs = pairs
        self.abf = abf_list[0]
        self.fig.canvas.toolbar_position = 'bottom'
        self.lowerline = None
        self.upperline = None
        self.abf_plot.grid(True)
        #Starting vars
        self.sweep_list = [0,5,14]
        self.lowerlim = 0
        self.upperlim = 1
        self.maxinterval = 0.05
        self.minheight = 5
        self.minpeak = -10
        self.threshfrac = 5
        self.xlimit = [0, 2]
        self.alim = [0,2]
        self.dvdt = 20
        self.sweep_data = [None, None, None]
        self.sweepwise_spikes = [None, None, None]
        self.sweepwise_spikes_plot = [None, None, None]
        self.dvdt_line = None
        # define widgets
        self.dir = widgets.Text(
                value='.',
                placeholder='.',
                description='ABF DIR:',
            continuous_update=False,
                disabled=False
            )
        self.abf_file = widgets.Dropdown(
            options=self.pairs,
            description='Number:',
        )
        self.sweeps = widgets.Text(
                value='0, 0',
                placeholder='0, 0',
                description='Sweeps:',
            continuous_update=False,
                disabled=False
            )
        xanalysis = widgets.FloatRangeSlider(
                value=[0, 2],
                min=0,
                max=self.abf.sweepX[-1],
                step=0.1,
                description='Analysis range:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
            )
        xlim = widgets.FloatRangeSlider(
                value=[0, 2],
                min=0,
                max=self.abf.sweepX[-1],
                step=0.1,
                description='X limits:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
            )
        dvdt = widgets.FloatText(
            value=20.0,
            description='dV/dT:',
            disabled=False,
            style=textbox_style
        )
        Maxinterval = widgets.FloatText(
            value=5,
            description='max interval tp (ms):',
            disabled=False,
            style=textbox_style
        )
        minheight = widgets.FloatText(
            value=5.0,
            description='min height (mV):',
            disabled=False,
            style=textbox_style
        )
        minpeak = widgets.FloatText(
            value=-10.0,
            description='min peak (mV):',
            disabled=False,
            style=textbox_style
        )
        threshfrac = widgets.FloatText(
            value=5,
            description='thres frac (%):',
            disabled=False,
            style=textbox_style
        )
        self.run_folder = widgets.Button(
                description='Run Folder Analysis',
                disabled=False,
                button_style='', # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='check' # (FontAwesome names without the `fa-` prefix)
            )
        
 
        controls = widgets.VBox([self.dir,
                                 self.abf_file,
            self.sweeps,
            xlim,
            xanalysis,
            dvdt,
            Maxinterval,
            minheight,
            minpeak,
            threshfrac,
            self.run_folder,
        ])
        controls.layout = make_box_layout()
        self.df_out = widgets.Output()
        out_box = widgets.VBox([output, self.df_out])
        output.layout = make_box_layout()
        self.df_out.layout = make_box_layout()
        
        
        
        # observe stuff
        self.dir.observe(self.update_abf_list, 'value')
        self.abf_file.observe(self.update_abf, 'value')
        xanalysis.observe(self.parse_analysis_lim, 'value')
        xlim.observe(self.update_xlim, 'value')
        dvdt.observe(self.update_dvdt, 'value')
        self.sweeps.observe(self.parse_sweeps, 'value')
        Maxinterval.observe(self.update_max_int, 'value')
        minheight.observe(self.update_min_height, 'value')
        minpeak.observe(self.update_min_peak, 'value')
        threshfrac.observe(self.update_threshfrac, 'value')
        self.run_folder.on_click(self.run_folder_analysis)
        self.plot_abf()
        self.update_dvdt_line
        self.run_analysis()
        self.plot_analysis()
        # add to children
        self.children = [controls, out_box]
     
        
    def update_lower_line(self, lwlim):
        self.lowerlim = lwlim
        if self.lowerline is not None:
            self.lowerline.set_xdata([self.lowerlim, self.lowerlim])
        else:
            plot = self.abf_plot
            self.lowerline = plot.axvline(self.lowerlim)
    def update_upper_line(self, uplim):
        self.upperlim = uplim
        if self.upperline is not None:
            self.upperline.set_xdata([self.upperlim, self.upperlim])
        else:
            plot = self.abf_plot
            self.upperline = plot.axvline(self.upperlim)
            
    def parse_analysis_lim(self, change):
            self.alim = change.new
            self.update_lower_line(self.alim[0])
            self.update_upper_line(self.alim[1])
            self.update_dvdt_line()
            self.run_analysis()
            self.plot_analysis()
        
    def update_dvdt(self, change):
        self.dvdt = change.new
        self.update_dvdt_line()
        self.run_analysis()
        self.plot_analysis()
    
    def update_abf_list(self, change):
        abf_dir = change.new
        self.abf_list = glob.glob(abf_dir + "\\**\\*.abf", recursive=True)
        self.abf_list_name = [os.path.basename(x) for x in self.abf_list]
        self.pairs = [c for c in zip(self.abf_list_name, self.abf_list)]
        self.abf_file.options = self.pairs
        
    def update_dvdt_line(self):
        if self.dvdt_line is not None:
            self.dvdt_line.set_ydata([self.dvdt, self.dvdt])
        else:
            plot = self.dvdt_plot
            self.dvdt_line = plot.axhline(self.dvdt)
        
    def run_analysis(self):
        spikext = feature_extractor.SpikeFeatureExtractor(filter = 0, dv_cutoff=self.dvdt, start=self.alim[0], 
                                                          end=self.alim[1], max_interval=self.maxinterval,
                                                         min_height=self.minheight, min_peak=self.minpeak,
                                                         thresh_frac=self.threshfrac)
        for i,x in enumerate(self.sweep_list):
            self.abf.setSweep(x)
            dataT, dataV, dataI = self.abf.sweepX, self.abf.sweepY, self.abf.sweepC
            spikes_df = spikext.process(dataT, dataV, dataI)
            spikes_df['sweep'] = np.full(len(spikes_df.index.values), x)
            spikes_df = spikes_df.set_index('sweep')
            self.sweep_data[i] = spikes_df
        for i, x in enumerate(self.sweep_data):
            if x.empty == False:
                spikes_t = x['peak_t'].to_numpy()
                spikes_v = x['peak_v'].to_numpy()
                self.sweepwise_spikes[i] = np.vstack([spikes_t, spikes_v])
            else:
                self.sweepwise_spikes[i] = None
                try:
                    self.sweepwise_spikes_plot[i].remove()
                    self.sweepwise_spikes_plot[i] = None
                except:
                    self.sweepwise_spikes_plot[i]
        with self.df_out:
            clear_output()
            temp = pd.concat(self.sweep_data)
            display(temp)
            
    def plot_analysis(self):
        plot = self.abf_plot
        for i, x in enumerate(self.sweepwise_spikes):
            try:
                    self.sweepwise_spikes_plot[i].remove()
                    self.sweepwise_spikes_plot[i] = None
            except:
                    self.sweepwise_spikes_plot[i]
            if x is not None:
                self.sweepwise_spikes_plot[i] = plot.scatter(x[0,:],x[1,:])
    
    def update_abf(self, change):
        self.abf = pyabf.ABF(change["new"])
        self.plot_abf()
        self.run_analysis()
        self.plot_analysis()
        
    def parse_sweeps(self, change):
        try:
            self.sweep_list = np.fromstring(change.new, dtype=int, sep=',')
        except:
            self.sweep_list = [0]
        self.sweep_data = [None for x in self.sweep_list]
        self.sweepwise_spikes = np.copy(self.sweep_data).tolist()
        self.sweepwise_spikes_plot = np.copy(self.sweep_data).tolist()
        
        self.plot_abf()
        self.run_analysis()
        self.plot_analysis()
        
    def plot_abf(self):
        """Draw line in plot"""
        self.fig.clear()
        self.upperline, self.lowerline = None, None
        self.abf_plot, self.dvdt_plot = self.fig.subplots(nrows=2, ncols=1, sharex=True)
        for x in self.sweep_list:
            #ax = self.fig.add_axes(figsize=(5, 3.5)sharex=True, sharey=True)
            self.abf.setSweep(x)
            self.abf_plot.plot(self.abf.sweepX, self.abf.sweepY)
            self.dvdt_plot.plot(self.abf.sweepX[:-1], (np.diff(self.abf.sweepY)/np.diff(self.abf.sweepX))/1000)
            self.update_dvdt_line()
        self.fig.gca().set_xlim(self.xlimit)
        
        self.fig.canvas.draw()
        self.update_lower_line(self.alim[0])
        self.update_upper_line(self.alim[1])
        
    def update_xlim(self, change):
        try:
            self.xlimit = change.new
            self.fig.gca().set_xlim(self.xlimit)
        except:
            return      
    
    def update_max_int(self, change):
        self.maxinterval = change.new/1000
        self.run_analysis()
        self.plot_analysis()
    
    def update_min_height(self, change):
        self.minheight = change.new
        self.run_analysis()
        self.plot_analysis()
        
    def update_min_peak(self, change):
        self.minpeak = change.new
        self.run_analysis()
        self.plot_analysis()
    
    def update_threshfrac(self, change):
        self.threshfrac = change.new
        self.run_analysis()
        self.plot_analysis()

    def run_folder_analysis(self, f):
        self.run_folder.disabled = True
        protocol = self.abf.protocolPath.split("\\")[-1].split(".")[0]
        
        self.run_folder.description = f'Running for {protocol}'
        param_dict = dict(filter = 0, dv_cutoff=self.dvdt, start=self.alim[0], 
                                                          end=self.alim[1], max_interval=self.maxinterval,
                                                         min_height=self.minheight, min_peak=self.minpeak,
                                                         thresh_frac=self.threshfrac)
        
        df = abfe.folder_feature_extract(os.path.abspath(str(self.dir.value)), param_dict=param_dict, protocol_name=str(protocol))
        print(df[0].columns.values)
        abfe.save_data_frames(*df, root_fold=self.dir.value)
        self.run_folder.disabled = False
        self.run_folder.description = 'Run Folder Analysis'
        pass
         
abfanalysis()

abfanalysis(children=(VBox(children=(Text(value='.', continuous_update=False, description='ABF DIR:', placehol…

INFO:root:Curve fit for membrane time constant failed


20o01023.abf import


INFO:root:Curve fit for membrane time constant failed
INFO:root:Curve fit for membrane time constant failed
C:\Users\SMest\Anaconda3\lib\site-packages\pyAPisolation\patch_utils.py:79: RuntimeWarning: Mean of empty slice
  data = np.nanmean(data)


Processed Sweep 005 with 3 aps
Processed Sweep 006 with 7 aps
Processed Sweep 007 with 10 aps


...

Processed Sweep 004 with 4 aps
Processed Sweep 005 with 8 aps
Processed Sweep 006 with 12 aps
Processed Sweep 007 with 14 aps


INFO:root:Need to recalculate threshold-peak pair that exceeds maximum allowed interval (0.050000 s)


Processed Sweep 008 with 17 aps
Processed Sweep 009 with 11 aps
Processed Sweep 010 with 4 aps
Processed Sweep 011 with 3 aps


C:\Users\SMest\Anaconda3\lib\site-packages\pyAPisolation\patch_utils.py:79: RuntimeWarning: Mean of empty slice
  data = np.nanmean(data)
C:\Users\SMest\Anaconda3\lib\site-packages\pyAPisolation\patch_utils.py:79: RuntimeWarning: Mean of empty slice
  data = np.nanmean(data)


Processed Sweep 012 with 3 aps
Processed Sweep 013 with 3 aps
Processed Sweep 014 with 3 aps
Processed Sweep 015 with 3 aps
plot failed
['__fold_name' 'adapt' 'adp_i' ... 'upstroke_t' 'upstroke_v' 'width']
